
# Training a model

***Zeroing-out gradients***

It is beneficial to zero out gradients when building a neural network.
This is because by default, gradients are accumulated in buffers (i.e,
not overwritten) whenever the `.backward()` method is called.

To track `torch.Tensor` (that composes a neural net) operations set the `.requires_grad` attribute as `True`.This happens on subsequent backward passes. The gradient for this tensor will be accumulated into `.grad`
attribute. The accumulation (or sum) of all the gradients is calculated
when the `.backward()` method is called on the __loss tensor__.

When starting a training loop, the gradients should be zeroed
out so that tracking is performed correctly.

## Walk-through

Training a Neural Net follows the common procedures:

1. Load and normalize the dataset; in PyTorch we'll also need to set a `Dataloader` to iterate over data
2. Build the neural networ Build the neural network
3. Define the loss function
4. Set the optimezer - zero-out the gradients while training the network

In [ ]:
import torch

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

import torchvision
# Transforms are common image transformations. They can be chained together using Compose.
import torchvision.transforms as transforms

## 1 - Load adn Normalize data

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                               ])
# get train data
trainset = torchvision.datasets.CIFAR10(root='./data',
                                        train=True,
                                        download=True,
                                        transform=transform)
# set DataLoader object for training
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=4,
                                          shuffle=True,
                                          num_workers=2)

# get train data
testset = torchvision.datasets.CIFAR10(root='./data',
                                       train=False,
                                       download=True,
                                       transform=transform)
# set data DataLoader object for test
testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=4,
                                         shuffle=False, # no need to shuffle data in test
                                         num_workers=2)

# set class tuple
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## 2 - Build the Neural Network

 A Neural Net is defined as a class that extends the `nn.Module` and implement
1. the `__init__` method
2. the `.forward` method

In [ ]:
class Net(nn.Module):
    # define neural net's architecture 
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    # define forward pass
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## 3 - Define  Loss function and Optimizer

In [ ]:
net = Net() # init network
criterion = nn.CrossEntropyLoss() # categorical cross-entropy loss for classification
# Set Stochastic Gradient Descent
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) # track model's .parameters()

## 5 - Zero the gradients while training the network

Iterate over the `DataLoader` (iterator), and feed the inputs to the network and optimize.

Gradients are zeroed-out every iteration (a.k.a. mini-batch gradient descent)

In [ ]:
n_epochs = 2 # more epoch wil output better performance
for epoch in range(n_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):

        inputs, labels = data

        # Zero-out gradients
        optimizer.zero_grad() # can use model.zero_grad() alternatively if params in optimizer

        # forward-pass
        outputs = net(inputs)
        # compute loss
        loss = criterion(outputs, labels)
        # backprop
        loss.backward()
        # optimize - apply gradient descent
        optimizer.step()
        
        # accumulate loss
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('(epoch: %d | batch: %5d) loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

# 6 - Inference (extra)

To use a Neural Network and perform some inference we do apply only the forward-pass. So we  don't need to track the network's parameters, so we run the model inside a `torch.no_grad` context that sets the `.require_grad` attribute to `False`.

In [ ]:
correct = 0
total = 0

# Set require_grad to False
with torch.no_grad():
    for X, y in testloader:
        output = net(X)
        for idx, i in enumerate(output):
            if torch.argmax(i) == y[idx]:
                correct += 1
            total += 1

print("Accuracy: %.2f" % (correct/total))

In [ ]:
net